
<img src="img/logo_wiwi.png" width="25%" align="left">

<img src="img/decision_analytics_logo.png" width="17%" align="right">



<br><br><br><br><br><br><br><br>



# Algorithmen und Datenstrukturen(A+D)-Projekt 

**Sommersemester 2023**


# 5. Informierte Suche mit Rollout als heuristischer Funktion
<br>

<br>
<br>

**J-Prof. Dr. Michael Römer, Till Porrmann**

Juniorprofessur für Decision Analytics  | Universität Bielefeld

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from numba import njit
import networkx as nx
import keyboard
from IPython.display import SVG, display, clear_output

from heapq import nsmallest

## Termine Zwischenpräsentation

**Zwischenpräsentation**: Terminvorschlag: Montag,  22. Mai 2023 von 9-12 Uhr

**Frage:**
- wem würde **kein Zeitslot** passen?

## Was machen wir heute?

- wir wiederholen Beam Search, und schauen, wie wir Beam Search auf das Rucksack-Problem anwenden können
- wir betrachten die so genannte **informierte Suche** 
- insbesondere nutzen wir **Rollout** als heuristische Funktion
- wir lernen eine Technik kennen, um "teure" Heuristiken wie Rollout nur begrenzt anzuwenden

- wir haben noch etwas Zeit, um Fragen mit den Gruppen zu klären

# Wiederholung: Nearest Neighbor als Greedy-Verfahren für das TSP

## Ein Greedy-Ansatz für das TSP / SHPP: Nearest-Neighbor

<img src="./img/32.png" width="20%" align="right">


Eine einfache und sehr bekannte Heuristik für das SHPP (und das TSP) nennt sich **nearest neighbor**:
- man startet bei einem Knoten und
- wählt in jedem Schritt immer den Knoten aus, der dem aktuellen am nächsten ist:

Beispiel rechts: wir starten in Marin und suchen immer den nächsten Nachbarn

## TSP in Python: Distanzmatrix

- die einzige Information, die wir für die Darstellung des TSP brauchen, ist eine **Distanz-Matrix**
- in dieser (symmetrischen) Matrix steht in jedem Element $[i,j]$ die Distanz zwischen den Knoten (Orten) $i$ und $j$
- als Beispiel betrachten wir folgende Matrix:


In [5]:
distance_matrix = np.array([
    [0,  5, 4, 10],
    [5,  0, 8,  5],
    [4,  8, 0,  3],
    [10, 5, 3,  0]
])

distance_matrix_small = distance_matrix

## Eine Routine zum Evaluieren einer Lösung

- immer wenn man nicht-triviale Algorithmen für Optimierungsprobleme entwickelt, sollte man einen "Solution-Checker" schreiben / nutzen

><div class="alert alert-block alert-info">
<b>Was ist im Fall des TSP zu prüfen?</b></div>  


**Im Fall des TSP gilt es:**
- zu prüfen, ob
  - die Lösung die Richtige Anzahl an Knoten enthält
  - dass es sich bei der Lösung tatsächlich um eine Permutation der Indizes handelt (kein Index kommt zweimal vor)
- die Distanz der Tour zu berechnen

In [6]:
def evaluate_tsp_solution(distance_matrix, tour):
    n = len(distance_matrix)
    if len(tour) != n:
        print ("Wrong number of nodes")
        return -1
     # Menge der Lösungsindizes muss = der Menge der Indizes von 0 bis n-1 sein
    if set(tour) != set(range(n)):
        print ("Not a proper tour!")
        return  -1
    
    total_distance = 0
    for i in range(n):
        if i < n-1:
            total_distance += distance_matrix[tour[i],tour[i+1]]
        else:
            total_distance += distance_matrix[tour[i],tour[0]]   
            
    return total_distance    

...probieren wir es aus:

In [7]:
tour = [0,1,2,3]
evaluate_tsp_solution(distance_matrix, tour)

26

## Erweiterung: Evaluation zum Überprüfen einer gegebenen Distanz

- oftmals ist es nützlich, in einer Funktion direkt die vom Algorithmus berechnete Distanz zu prüfen
- folgende Funktion macht eine entsprechende Ergebnisausgabe:


In [8]:
def print_obj_and_eval_tsp_solution(distance_matrix, tour, distance):
    
    eval_distance = evaluate_tsp_solution(distance_matrix, tour)
    
    if distance == eval_distance:
        print ("Solution feasible, distance is: ", distance)
    elif eval_distance < 0:
        print("Solution infeasible")
    else: 
        print("Solution feasible, wrong distance: ", distance, " evaluation gave ", eval_distance)

...probien wir es aus:

## Das Python-Paket `python-tsp` testen

- TSP-Instanzen einlesen

In [9]:
from python_tsp.distances import tsplib_distance_matrix

# Lokale Installation
instances_path = "./../instances/tsp/" 

#Im Fall von Colab (oder lokal im selben Ordner wir Notebook) nach drag&drop:
#instances_path = "./" 

#instance_name = "a280.tsp" # optimale Lösung 2579 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
#instance_name = "brazil58.tsp" # optimale Lösung 25395 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
instance_name = "berlin52.tsp" # optimale Lösung 7542 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
tsplib_file = instances_path+instance_name 
distance_matrix = tsplib_distance_matrix(tsplib_file)

instance_name_medium = "ulysses22.tsp" # optimale Lösung 7013(lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
tsplib_file_medium = instances_path+instance_name_medium 
distance_matrix_medium = tsplib_distance_matrix(tsplib_file_medium)

 - heuristische TSP-Algorithmen:

In [10]:
from python_tsp.heuristics import solve_tsp_local_search, solve_tsp_simulated_annealing

In [11]:
%%time
tour, distance = solve_tsp_simulated_annealing(distance_matrix)
distance

CPU times: total: 1.22 s
Wall time: 1.22 s


7881

- exakte TSP-Algorithmen basierend auf Enumeration und dynamischer Programmierung
  - (vorsicht: dauert wahrscheinlich lange mit Instanzen > 10 Items)
  

In [12]:
from python_tsp.exact import solve_tsp_dynamic_programming

#tour, distance = solve_tsp_dynamic_programming(distance_matrix_21)

#distance

## Nearest Neighbor als Greedy-Verfahren für das TSP: Hilfsfunktion

- hier eine etwas andere Implementierung als zuvor:
  - zuvor hatten wir eine Hilfsfunktion: `select_nearest_neighbor`, die den nächsten Knoten ausgewählt hat
  - nun schreiben wir eine Funktion, die uns für jeden zulässigen Nachbarn (der noch nicht in der Tour ist) ein `tuple` `(distance, tour)` gibt, mit der Distanz und der gesamten Kandidaten-Tour 
  - beachte: Wir schließen die "Rückfahrt" ein!

In [13]:
@njit
def get_dist_feasible_candidate_tours(tour, total_distance, distance_matrix):
    
    dist_candidate_tour = []
    for neighbor, distance in enumerate(distance_matrix[tour[-1]]):
        if neighbor in tour:
            continue
        
        candidate_tour = tour + [neighbor]
        
        if len(candidate_tour) == len(distance_matrix):
            distance += distance_matrix[neighbor, tour[0]]
        
        total_distance_candidate_tour = total_distance + distance
        dist_candidate_tour.append((total_distance_candidate_tour, candidate_tour))
        
        
    return dist_candidate_tour


In [61]:
dist_tour = get_dist_feasible_candidate_tours([1], 0, distance_matrix_small)
print (dist_tour)
min(dist_tour)

[(5, [1, 0]), (8, [1, 2]), (5, [1, 3])]


(5, [1, 0])

## Nearest Neighbor als Greedy-Verfahren für das TSP: Hauptfunktion

- die Hauptfunktion `tsp_greedy` sieht dann auch leicht anders aus:
  - beachte: der erste Parameter ist eine Tour!
  - und: durch die Verwendung des tupels `(distance, tour)` für jeden Nachbarn können wir einfach das Minimum aus der Liste der Tupel suchen (denn das erste Element wird zum Vergleich zuerst herangezogen)

In [15]:
@njit
def tsp_greedy(tour, distance_matrix):
        
    total_distance = 0
    
    #solange die sequenz noch nicht alle Knoten umfasst
    while len(tour) < len(distance_matrix):    
        
        #bestimme alle zulässigen Nachbarn und deren Entfernungen
        dist_candidate_tours = get_dist_feasible_candidate_tours(tour, total_distance, distance_matrix)   
        
        # bestimme das minimale Tupel (der erste Wert des Tupels, d.h. die Distance, wird automatisch genutzt)
        distance_best_candidate_tour, best_candidate_tour = min(dist_candidate_tours)
        
        tour = best_candidate_tour
        total_distance = distance_best_candidate_tour    
        

    return tour, total_distance

In [60]:
tour, distance = tsp_greedy([0], distance_matrix)
distance

8980

## Abstraktion: Zustands(raum)modellierung / State Space Representation

Ein Greedy-Verfahren konstruiert Schritt für Schritt eine Lösung (es ist ein Konstruktionsverfahren)

Für jeden Schritt müssen wir folgende Fragen beantworten:
1. Welche Entscheidungen / Handlungen / Aktionen sind möglich?
2. Welche der möglichen Entscheidungen sollte gewählt werden?

Zur Beantwortung von Frage 1 brauchen wir:
- eine Beschreibung des aktuellen **Zustands**:
  - TSP: die bisherige Tour
  - Rucksackproblem: das bisher eingepackte Gewicht 
- die "Regeln" / Restriktionen, die beschreiben, **welche Handlungen in einem Zustand zulässig sind**
  - TSP: kein erneuter Besuch einer bereits besuchten Stadt
  - Rucksackproblem: Gegenstand einpacken geht nur, wenn das Gesamtgewicht nicht überschritten wird

Zur Beantwortung von Frage 2 brauchen wir ein Auswahl-Kriterium
- z.B. der Wert der neuen Teillösung:
  - TSP: Distanz der (Teil-)Tour nach Hinzunahme des Nacharn
  - Rucksackproblem: Gesamtwert der Lösung nach Hinzunahme eines Elements

## Die Suche im Lösungsraum kann als Graph dargestellt werden

- man nennt diesen Graph den Suchgraph oder auch **Suchbaum**
- in diesem Graph entsprechen 
  - die Knoten den **Zuständen** (siehe oben)
  - die Kanten den **Entscheidungen**
  


Wir können daher **Suchverfahren in Graphen** anwenden, um eine optimale Lösung zu finden.
- wir schauen uns das gleich einmal für den Greedy-Fall an

## Beachte: Es folgen Funktionen zur Visualisierung
- wir möchten manche unserer Algorithmen visualisieren
- dazu nutzen wir einige Hilfsfunktionen sowie
- angepasste Implementierungen unserer Algorithmen

**Wichtig:** Die Anpassungen brauchen wir nicht für die Algorithmen selbst!

In [17]:
    
def add_to_search_graph(g, prev_state, state, decision, distance_decision, distance_state):
    clear_output()
    g.add_edge(prev_state,state, label= " add:" + str(decision) + " d:" + str(distance_decision))
    g.nodes()[state]["xlabel"]= distance_state
    svg = nx.nx_agraph.to_agraph(g).draw(prog='dot',format='svg')
    display(SVG(svg))


## Eine visualisierte Version des Greedy-Verfahrens für das TSP

In [18]:
def tsp_greedy_visualized(tour, distance_matrix):
    total_distance = 0
    g = nx.MultiDiGraph()
    
    #solange die sequenz noch nicht alle Knoten umfasst
    while len(tour) < len(distance_matrix):    
        
        #bestimme alle zulässigen Kandidatentouren und deren Entfernungen
        dist_candidate_tours = get_dist_feasible_candidate_tours(tour, total_distance, distance_matrix)   
        
        # bestimme das minimale Tupel (der erste Wert des Tupels, d.h. die Distance, wird automatisch genutzt)
        distance_best_candidate_tour, best_candidate_tour = min(dist_candidate_tours)
        
        next_node = best_candidate_tour[-1]
        
        next_node_distance = distance_best_candidate_tour - total_distance
        
        add_to_search_graph(g, tuple(tour), tuple(best_candidate_tour), next_node, next_node_distance, total_distance)
        
        tour = best_candidate_tour
        total_distance = distance_best_candidate_tour   
        keyboard.read_key()
        
    return tour, total_distance    


In [19]:

#tour, distance = tsp_greedy_visualized([0], distance_matrix_small)
print (distance)

8980


## Durchlaufen des Suchbaums: Breitensuche

- Greedy betrachtet pro Stufe / Level / Ebene des Suchbaums nur einen einzigen Zustandsknoten
- Wenn wir alle Knoten betrachen, haben wir eine **Breitensuche** (siehe Vorlesung)

In [20]:
def tsp_breadth_first_search_visualized(tour, distance_matrix):
        
    total_distance = 0
    
    ## für die Visualisierung
    g = nx.MultiDiGraph()
    
    # alle Zustandsknoten als Distanz-Tour-Tupel in der aktuellen Stufe / Ebene 
    dist_tours_current_stage = [(0, tour.copy())] 
    
    for stage in range(0,len(distance_matrix)-1):
        
        dist_tours_next_stage = []
        
        for tour_dist, current_tour in dist_tours_current_stage:
            
            old_tour_tuple = tuple(current_tour)
            
            #bestimme alle zulässigen Tourerweiterungen und deren Entfernungen
            dist_candidate_tours = get_dist_feasible_candidate_tours(current_tour, tour_dist, distance_matrix)   
        
            for candidate_tour_dist, candidate_tour in dist_candidate_tours:
                         
                next_node = candidate_tour[-1]
        
                next_node_distance = candidate_tour_dist - tour_dist
            
                add_to_search_graph(g, old_tour_tuple, tuple(candidate_tour), next_node, next_node_distance, candidate_tour_dist)
                
                keyboard.read_key()

                dist_tours_next_stage.append( (candidate_tour_dist, candidate_tour) )
                
        dist_tours_current_stage = dist_tours_next_stage.copy()

    best_distance, best_tour  = min(dist_tours_current_stage)
    return best_tour, best_distance

In [21]:
distance_matrix_5 =  np.array([
    [0,  5, 4, 10, 8],
    [5,  0, 8,  5, 4],
    [4,  8, 0,  3, 7],
    [10, 5, 3,  0, 9 ],
    [8,  4, 7,  9, 0]
])

#tour, distance = tsp_breadth_first_search_visualized([0], distance_matrix_5)

In [22]:
print(distance)

8980


# Beam Search: Einschränkung der Breitensuche

- Idee: ein Kompromiss zwischen Greedy und Breitensuche
- übernimm auf jeder Stufe nur eine bestimmte Anzahl (beam-width) von Zustandsknoten
  - Auswahl z.B. auf Basis von Distanz der Tour
- hier einfache Implementierung mit der Funktion `nsmallest` aus dem Paket `heapq`

In [23]:
from heapq import nsmallest
def tsp_beam_search_visualized(tour, distance_matrix, beam_width):
        
    total_distance = 0
    
    ## für die Visualisierung
    g = nx.MultiDiGraph()
    
    # alle Zustandsknoten als Distanz-Tour-Tupel in der aktuellen Stufe / Ebene 
    dist_tours_current_stage = [(0, tour.copy())] 
    
    for stage in range(0,len(distance_matrix)-1):
        
        dist_tours_next_stage = []
        
        for tour_dist, current_tour in dist_tours_current_stage:
            
            old_tour_tuple = tuple(current_tour)
            
            #bestimme alle zulässigen Tourerweiterungen und deren Entfernungen
            dist_candidate_tours = get_dist_feasible_candidate_tours(current_tour, tour_dist, distance_matrix)     
            
        
            for candidate_tour_dist, candidate_tour in dist_candidate_tours:
                         
                next_node = candidate_tour[-1]
        
                next_node_distance = candidate_tour_dist - tour_dist
            
                add_to_search_graph(g, old_tour_tuple, tuple(candidate_tour), next_node, next_node_distance, candidate_tour_dist)
                
                keyboard.read_key()

                dist_tours_next_stage.append( (candidate_tour_dist, candidate_tour) )
                
        dist_tours_current_stage = nsmallest(beam_width, dist_tours_next_stage)

    
    best_distance, best_tour  = min(dist_tours_current_stage)
    
    return best_tour, best_distance

In [24]:
#tour, distance = tsp_beam_search_visualized([0], distance_matrix_5 1)

In [25]:
print(distance)

8980


## Beam Search ohne Visualisierung

- zum Testen größerer Instanzen hier Code ohne Visualisierung:

In [26]:

def tsp_beam_search(tour, distance_matrix, beam_width):
        
    total_distance = 0
    
    
    # alle Zustandsknoten als Distanz-Tour-Tupel in der aktuellen Stufe / Ebene 
    dist_tours_current_stage = [(0, tour.copy())] 
    
    for stage in range(0,len(distance_matrix)-1):
        
        dist_tours_next_stage = []
        
        for tour_dist, current_tour in dist_tours_current_stage:
                  
            #bestimme alle zulässigen Tourerweiterungen und deren Entfernungen
            dist_candidate_tours = get_dist_feasible_candidate_tours(current_tour, tour_dist, distance_matrix)     
            
        
            for candidate_tour_dist, candidate_tour in dist_candidate_tours:                         

                dist_tours_next_stage.append( (candidate_tour_dist, candidate_tour) )
                
        dist_tours_current_stage = nsmallest(beam_width, dist_tours_next_stage)
    
    best_distance, best_tour  = min(dist_tours_current_stage)
    return best_tour, best_distance

..probieren wir es aus:

In [27]:
tour, distance = tsp_beam_search([0], distance_matrix, 1000)

print (distance)

8446


## Beobachtung: Eine größere Breite impliziert nicht unbedingt eine bessere Lösung! 

- intuitiv könnte man denken, dass eine breitere Suche automatisch zu einer besseren Lösung führt
- das stimmt auch im Grenzfall einer unbegrenzten Breite - da haben wir eine vollständige Enumeration
- allerdings kann eine Erweiterung der Breite auch verschlechternd wirken, siehe unser Beispiel für das TSP:

In [28]:
beam_widths = [1, 10, 50, 100, 1000]

for beam_width in beam_widths:    
    tour, distance = tsp_beam_search([0], distance_matrix, beam_width)
    print ("Beam width", beam_width, "Distanz:", distance)

Beam width 1 Distanz: 8980
Beam width 10 Distanz: 9423
Beam width 50 Distanz: 9497
Beam width 100 Distanz: 9846
Beam width 1000 Distanz: 8446


- das kann insbesondere dann passieren, wenn das Auswahl- bzw. Evaluationskritierium, nach dem die zu untersuchenden Zustände / Knoten im Suchbaum nicht sehr gut sind.

.. für eine Diskussion einschließlich eines Beispiels siehe z.B. https://cs.stackexchange.com/questions/117600/why-increasing-the-beam-width-does-not-necessarily-improve-the-beam-search-solut

- schauen wir später, wie wir an bessere Evaluationskriterien kommen!

# Übung: Beam Search für das Rucksackproblem



## Greedy-Algorithmus für das Rucksack-Problem: Implementierung in Python

In [29]:
def greedy_knapsack(values, weights, capacity):
    solution = [] # Array mit Lösungen
    obj_val=0 # akkumulierter Wert
    total_weight=0 # akkumuliertes Gewicht
    
    for i in range(len(values)): 
        if total_weight + weights[i] <= capacity: ## wenn das item noch reinpasst...
            solution.append(i) ## füge es hinzu und
            total_weight+= weights[i] # erhöhe das verbrauchte Gewicht
            obj_val += values[i] # und den Zielfunktionswert
    return obj_val, solution


.. probieren wir es aus:

In [30]:
values = [3000,2000,1500]
weights = [30,20,15]
capacity = 35

greedy_knapsack(values, weights, capacity)

(3000, [0])

## Verbesserungsidee Greedy-Algorithmus: Sortieren der Elemente

Welche Sortierungen wären sinnvoll?

Wie kann man eine solche Sortierung implementieren?

## Sortierung: Nutzung von vorhandenen Funktionen

- in "reinem" Python: `sorted(liste)` gibt die Liste 

In [31]:
sorted(weights)

[15, 20, 30]

- in `numpy` gibt es eine Reihe von Sortierfunktionen, z.B. `sort`

In [32]:
np.sort(weights)

array([15, 20, 30])

**Problem:** Wir haben jetzt zwar die sortierten Gewichte, aber wir möchten wissen, zu welchen Items / Gegenstände zu diesen gehören!

## Sortierung: Wie können wir beide Arrays "simultan" sortieren?

- in `numpy` gibt es die Funktion `argsort`, die nicht die sortierten Elemente zurückgibt, sondern die Indizes der Elemente:

In [33]:
sorted_indexes = np.argsort(weights)
sorted_indexes

array([2, 1, 0], dtype=int64)

- diese sortierten Indizes können zum umsortieren von `numpy`-arrays wie folgt genutzt werden:

In [34]:
values = np.array(values)
weights = np.array(weights)

values_sorted_by_weight = values[sorted_indexes]
weights_sorted_by_weight = weights[sorted_indexes]

weights_sorted_by_weight



array([15, 20, 30])

## Greedy-Knapack mit aufsteigend nach Gewicht sortiertem Array:

In [35]:
greedy_knapsack(values_sorted_by_weight, weights_sorted_by_weight, capacity)

(3500, [0, 1])

## Sortierung: Wie können wir die Arrays nach einem anderen Kriterium sortieren?

Mögliche Sortierungen wären:
- absteigend nach Wert
- absteigend nach Wert / Gewicht


**Einfacher Ansatz** zum Erhalt einer **absteigenden Sortierung**: Multiplikation mit -1

In [36]:
negative_values = values * -1
sorted_indexes = np.argsort(negative_values)

values_sorted = values[sorted_indexes]
weights_sorted = weights[sorted_indexes]
values_sorted

array([3000, 2000, 1500])

**Einfacher Ansatz** zum Erhalt einer **absteigenden Sortierung nach Wert / Gewicht**: Bildung eines Hilfsarrays

In [37]:
negative_ratio = values/weights * -1

sorted_indexes = np.argsort(negative_ratio)

values_sorted = values[sorted_indexes]
weights_sorted = weights[sorted_indexes]
values_sorted

array([3000, 2000, 1500])

## Genug Sandkasten-Beispiele: Größere Probleminstanzen

- eines der Ziele dieser Veranstaltung besteht darin, auch größere / schwierigere Probleme zu lösen
  - genauer gesagt: größere / schwierigere **Probleminstanzen**
- wir könnten solche Instanzen (zufällig) generieren
- es existieren jedoch für grundlegende kombinatorische Optimierungsprobleme in der Regel eine Reihe von bekannten Probleminstanzen, die z.B. in Form von strukturierten Textdateien vorliegen
- diese können z.B. zum Vergleich / Benchmarking von Algorithmen verwendet werden
- es gibt auch für viele schwere Probleme so genannte "offene" Instanzen, die bisher noch nicht optimal gelöst wurden

  


## Probleminstanzen für das Rucksackproblem

- für das Rucksackproblem gibt es sehr viele Instanzen
- ein bekannter Satz von Instanzen wurde von D. Pisinger erstellt
- wir haben ein paar dieser Instanzen im Instanz-Ordner zu diesem Foliensatz hinterlegt

Für einige dieser Instanzen findet man unter folgendem Link die optimalen Lösungen:

http://artemisa.unicauca.edu.co/~johnyortega/instances_01_KP/


## Dateiformat der Rucksack-Probleminstanzen

Die Instanzdateien haben folgendes Format:

- erste Zeile: `anzahl_items` `kapazität`
- jede weitere Zeile enthält für jedes Item: `wert` `gewicht`

Unser "Sandkastenbeispiel" sähe in diesem Format wie folgt aus:

## Einlesen von Probleminstanzen

Wollen wir die bekannten Instanzen nutzen, so müssen wir: 
- die Daten aus den Textdateien auslesen und
- in Datenstrukturen überführen, die wir für unsere Algorithmen nutzen können:

In [38]:
def read_knapsack_instance(filename):
    weights=[]
    values=[]
    with open(filename) as f: # öffne die Datei
        line = f.readline().split()  # splitte die erste Zeile in Einzelelemente
        number_of_items = int(line[0]) # lese die Anzahl items aus Zeile 1 aus
        capacity = int(line[1]) # lese die Kapazität aus
        for i in range(number_of_items): # lese die Werte der einzelnen Items aus
            line = f.readline().split() # lese Zeile aus Datei und splitte die Zeile in Einzelelemente
            values.append(int(line[0])) # lese den Wert
            weights.append(int(line[1])) # lese die Kapazität
    return np.array(values), np.array(weights), capacity

... probieren wir es aus an einer Instanz mit 5000 Items:

In [39]:
filename = "../instances/knapsack/knapPI_1_5000_1000_1"

values, weights, capacity  = read_knapsack_instance(filename)

## Lösen der Probleminstanzen

- nun können wir unseren Greedy-Ansatz und veschiedene Sortierungen evaluieren!

In [40]:
values, weights, capacity  = read_knapsack_instance(filename)

obj_val, solution = greedy_knapsack(values,weights,capacity)

print ("Zielfunktionswert in der Basisversion:", obj_val)

Zielfunktionswert in der Basisversion: 33727


.. wie gut ist die Lösung im Vergleich zum Optimum? Schauen wir nach:
http://artemisa.unicauca.edu.co/~johnyortega/instances_01_KP/

><div class="alert alert-block alert-info">
<b> Testen Sie den Greedy-Algorithmus mit verschiedenen Sortierungen und vergleichen Sie mit der optimalen Lösung!</b></div>


In [41]:
negative_ratio = values/weights * -1

sorted_indexes = np.argsort(negative_ratio)

values_sorted = values[sorted_indexes]
weights_sorted = weights[sorted_indexes]
values_sorted

obj_val, solution = greedy_knapsack(values_sorted,weights_sorted,capacity)

print ("Zielfunktionswert mit Ratio-Sortierung:", obj_val)


Zielfunktionswert mit Ratio-Sortierung: 276379


### Aufgabe: Entwickeln Sie einen Beam Search-Algorithmus für das Rucksack-Problem!

- wie lautet die grundlegende Idee?

- nutzen Sie den Greedy-Algorithmus für das Rucksack-Problem sowie die Beam-Search-Implementierung für das Rucksack-Problem als Basis!

- Beachten Sie: Wir haben es  mit einem Maximierungsproblem zu tun!
  - anstelle von `nsmallest` können Sie `nlargest` nutzen



In [42]:
from heapq import nlargest

def knapsack_beam_search(values, weights,  beam_width):
    
    return

# Informierte Suche

## Uninformierte Suche

- wie bereits mehrfach erwähnt, ist ein Greedy-Verfahren, das lediglich auf den Kosten der Teillösung durch eine Entscheidung (z.B. Länge der Tour nach Hinzufügen des Kandidaten / Nachbarn) bzw. auf den Kosten der Entscheidung (z.B. die Distanz zum Kandidaten / Nachbarn) **kurzsichtig**, weil es die zukünftigen Folgen der Entscheidung nicht berücksichtigt
- für **Beam Search** gilt natürlich das gleiche, weil ebenfalls ein Kriterium nutzen, um die weiter zu verfolgenden Zustände / Knoten im Suchbaum zu bestimmen
- in der Baumsuche nennt man diese Funktion, nach der man die Reihenfolge der Abarbeitung bestimmt, **Evaluationsfunktion**, die man als $f(n)$ schreibt, wobei $n$ ein Zustand bzw. Suchknoten ist.


- in kurzsichtigen oder auch **uninformierten** Verfahren nutzt man zur Auswahl lediglich die **Kosten der Teillösung** der offenene Suchknoten / Zustände, die man als $g(n)$ schreibt
  - d.h. $f(n) = g(n)$
  - das ist, was in unserem obigen Greedy bzw. in unserem obigen Beam Search macht

## Informierte Suche

- in der **informierten Suche** versucht man, die Auswirkungen der Entscheidung / Auswahl auf die Teillösung vom Suchknoten bis zum Ende der Suche (bis zu einer fertigen Lösung) abzuschätzen, man versucht also die **cost-to-go** abzuschätzen

- man nennt eine Funktion, mit der man diese Abschätzung machen kann, eine **heuristische Funktion**, die allgemein als $h(n)$ geschrieben wird


- diese Abschätzung der zukünftigen Kosten wird dann kombiniert mit den "bisherigen" Kosten, und man erhält die Evaluationsfunktion:

  - $f(n) = g(n) + h(n)$

- haben Sie eine Idee, was wir für das TSP als heuristische Funktion nutzen könnten?

## Greedy mit Rollout als Beispiel für die Nutzung einer heuristischen Funktion (für das TSP)

- wir haben bereits eine einfaches Beispiel für eine heuristische Funktion kennengelernt:
- **Rollout**, wobei eine Greedy-Algroithmus von jedem Kandidatenzustand bis zum Ende ausgeführt wird, um eine **Abschätzung** des zukünftigen Wertes jedes Kandidaten zu erhalten!

- die folgende Funktion erhält einen Zustand (d.h. eine Teil-Tour) und eine Distanzmatrix und gibt die Kosten den Greedy bi zum Ende zurück (beachte: die "bisherigen" Kosten fließen nicht ein!)

In [43]:
@njit
def heuristic_rollout(tour, distance_matrix):

    _, distance = tsp_greedy(tour, distance_matrix)

    return distance


Beispiel:

In [55]:
heuristic_rollout([0,1,2], distance_matrix)

9573

## Greedy mit einer heuristischen Funktion zur Auswahl

- wir stellen nun eine allgemeine Implementierung für die Verwendung einer heuristischen Funktion vor

- der letzte Parameter `heuristic function` ist eine **Funktion**, d.h. wir können eine beliebige Funktion nutzen
  - wir werden dann die Funktion `heuristic_rollout` übergeben

In [45]:
@njit
def tsp_greedy_with_heuristic(tour, distance_matrix, heuristic_function):
    
    total_distance = 0
    
    #solange die sequenz noch nicht alle Knoten umfasst
    while len(tour) < len(distance_matrix):    
        
        #bestimme für alle zulässigen Nachbarn die Teil-Touren deren Entfernungen
        dist_candidate_tours = get_dist_feasible_candidate_tours(tour, total_distance, distance_matrix)   
        
        # berechne für alle Kandidaten-Touren mit Hilfe der Heuristik den Wert f(n) = g(n)+h(n)
        # estimates_candidates_tours gibt uns für jede Kandidatentour ein Triple (f(n), g(n), tour), d.h. (estimate, dist, tour)    
        estimates_candidate_tours = get_estimates_candidate_tours(dist_candidate_tours, distance_matrix, heuristic_function)
    
        # bestimme das minimale Tupel (der erste Wert des Tupels, d.h. die Abschätzung f(x), wird automatisch genutzt)
        estimate_best_candidate_tour, dist_best_candidate_tour, best_candidate_tour = min(estimates_candidate_tours)
        
        tour = best_candidate_tour
        total_distance = dist_best_candidate_tour # wichtig: Hier nicht das estimate, sondern die Distanz (g(n))   
        
        
    return tour, total_distance

## Die Hilfsfunktion `get_estimates_candidate_tours`

..berechnet mit Hilfe einer heuristischen Funktion `heuristic_function` ($h(n)$) für jede Kandidatentour $n$ die geschätzte Gesamtlänge $f(n) = g(n) + h(n)$ 

Parameter: 
- Liste mit Tupeln `(dist, tour`) für alle Kandidaten
- Distanzmatrix 
- eine heuristische Funktion `heuristic_function`

Rückgabe:
- Liste mit 3-Tupeln (triples) `(estimate, dist, tour)` für jede Kandidatentour


In [46]:
@njit
def get_estimates_candidate_tours(dist_candidate_tours, distance_matrix, heuristic_function):
    
    estimates_candidate_tours = []
    for dist, candidate_tour in dist_candidate_tours:
        heuristic_value = heuristic_function(candidate_tour, distance_matrix)
        estimate = dist + heuristic_value
        estimates_candidate_tours.append((estimate, dist, candidate_tour))
        
    return estimates_candidate_tours

## Nun haben wir alles beisammen für ein "informiertes Greedy" / Greed mit Heuristik:

..probieren wir es aus!

In [59]:
tour, distance = tsp_greedy_with_heuristic([0], distance_matrix, heuristic_rollout)

print (distance)

8042


- wir haben nun also eine andere, etwas allgemeinere Implementierung von Greedy+Rollout
- betrachten wir nun eine informierte Variante von **Beam Search**

## Informiertes Beam Search 

- die Logik der heuristischen Funktion lässt sich natürlich auch auf Beam Search übertragen
- hier dient die informierte Evaluationsfunktion $f(n) + g(n) + h(n)$ als Auswahlkritirium für die Knoten, die in der nächsten Stufe weiter untersucht werden sollen:

In [48]:
@njit
def tsp_informed_beam_search(tour, distance_matrix, beam_width, heuristic_function):
        
    total_distance = 0
    
    # alle Zustandsknoten als Estimate-Distanz-Tour-Tupel in der aktuellen Stufe / Ebene 
    estimate_tours_current_stage = [(0, 0, tour)] 
    
    for stage in range(0,len(distance_matrix)-1):
        
        estimate_tours_next_stage = []
        
        for tour_estimate, tour_dist, current_tour in estimate_tours_current_stage:
                  
            #bestimme alle zulässigen Tourerweiterungen und deren Entfernungen
            dist_candidate_tours = get_dist_feasible_candidate_tours(current_tour, tour_dist, distance_matrix)  
            
            # berechne für alle Kandidaten-Touren mit Hilfe der Heuristik den Wert f(n) = g(n)+h(n)
            # estimates_candidates_tours gibt uns für jede Kandidatentour ein Triple (f(n), g(n), tour), d.h. (estimate, dist, tour)    
            estimates_candidate_tours = get_estimates_candidate_tours(dist_candidate_tours, distance_matrix, heuristic_function)
        
            for estimate_candidate_tour, dist_candidate_tour, candidate_tour in estimates_candidate_tours:                         

                estimate_tours_next_stage.append( (estimate_candidate_tour, dist_candidate_tour, candidate_tour) )
        
        # wähle die besten aus zur Betrachtung in der nächsten Stufe!
        estimate_tours_current_stage = nsmallest(beam_width, estimate_tours_next_stage)
        #print (stage, estimate_tours_current_stage)
        
    best_estimate, best_distance, best_tour  = min(estimate_tours_current_stage)
    return best_tour, best_distance

## Informiertes Beam Search: Test 

..probieren wir es aus:

In [56]:
tour, distance = tsp_informed_beam_search([0], distance_matrix, 40, heuristic_rollout)

print (distance)

7926


## Informierte Suche mit begrenzter Anwendung der Heuristik

#### Beobachtung:

- wenn man in jedem Suchknoten einmal einen kompletten Greedy **für jeden Nachbarn / Kandidaten** ausführt, dann ist das sehr aufwändig!
- insbesondere wird auch für "Nachbarn", die sehr weit weg liegen, trotzdem der Rollout durchgführt

#### Idee: Limitiere die Anwendung der Heuristik auf "vielversprechende Fälle"
- wir führen einen Parameter `no_candidates_for_heuristic` ein, der besagt, wie viele Nachbarn maximal mittels Heuristik betrachtet werden solln
- und betrachten nur die `no_candidates_for_heuristic` Nachbarn / Kandidaten, die nach dem Kriterium $g(n)$ (hier: Distanz der Teiltour bis zum Kandidaten) am besten sind:

#### Anmerkung:

Bertsekas nennt diese Idee im Kontext von Rollout  **simplified rollout** (siehe A+D-Projekt vom letzten Jahr)


In [50]:
@njit
def get_estimates_candidate_tours_limited(dist_candidate_tours, distance_matrix, heuristic_function, no_candidates_for_heuristic):
    
    estimates_candidate_tours = []
    
    # hier erfolgt die Auswahl der zu betrachtenden Kandidaten anhand der Distanz 
    # beachte: gleiche logik mit nsmallest wie schon oben
    dist_candidate_tours_limited = nsmallest(no_candidates_for_heuristic, dist_candidate_tours )
    
    for dist, candidate_tour in dist_candidate_tours_limited:
        heuristic_value = heuristic_function(candidate_tour, distance_matrix)
        estimate = dist + heuristic_value
        estimates_candidate_tours.append((estimate, dist, candidate_tour))
        
    return estimates_candidate_tours

## Begrenzte Anwendung der Heuristik bei Greedy

In [51]:
@njit
def tsp_greedy_with_heuristic_limited(tour, distance_matrix, heuristic_function, no_candidates_for_heuristic):
    
    total_distance = 0
    
    #solange die sequenz noch nicht alle Knoten umfasst
    while len(tour) < len(distance_matrix):    
        
        #bestimme alle zulässigen Nachbarn und deren Entfernungen
        dist_candidate_tours = get_dist_feasible_candidate_tours(tour, total_distance, distance_matrix)   
        
        
        estimates_candidate_tours = get_estimates_candidate_tours_limited(dist_candidate_tours, distance_matrix, heuristic_function, no_candidates_for_heuristic)
    
        # bestimme das minimale Tupel (der erste Wert des Tupels, d.h. die Distance, wird automatisch genutzt)
        estimate_best_candidate_tour, dist_best_candidate_tour, best_candidate_tour = min(estimates_candidate_tours)
        
        tour = best_candidate_tour
        total_distance = dist_best_candidate_tour    
        
        
    return tour, total_distance

..Test:

In [57]:
tour, distance = tsp_greedy_with_heuristic_limited([0], distance_matrix, heuristic_rollout, 10)

print (distance)

8122


## Begrenzte Anwendung der Heuristik in Beam Search

In [53]:
@njit
def tsp_informed_beam_search_limited(tour, distance_matrix, beam_width, heuristic_function, no_candidates_for_heuristic):
        
    total_distance = 0
    
    # alle Zustandsknoten als Estimate-Distanz-Tour-Tupel in der aktuellen Stufe / Ebene 
    estimate_tours_current_stage = [(0, 0, tour)] 
    
    for stage in range(0,len(distance_matrix)-1):
        
        estimate_tours_next_stage = []
        
        for tour_estimate, tour_dist, current_tour in estimate_tours_current_stage:
                  
            #bestimme alle zulässigen Tourerweiterungen und deren Entfernungen
            dist_candidate_tours = get_dist_feasible_candidate_tours(current_tour, tour_dist, distance_matrix)  

            estimates_candidate_tours = get_estimates_candidate_tours_limited(dist_candidate_tours, distance_matrix, heuristic_function, no_candidates_for_heuristic)
            
            for estimate_candidate_tour, dist_candidate_tour, candidate_tour in estimates_candidate_tours:                  
                estimate_tours_next_stage.append( (estimate_candidate_tour, dist_candidate_tour, candidate_tour) )
              
        estimate_tours_current_stage = nsmallest(beam_width, estimate_tours_next_stage)

    best_estimate, best_distance, best_tour  = min(estimate_tours_current_stage)
    return best_tour, best_distance

...Test:

In [58]:
tour, distance = tsp_informed_beam_search_limited([0], distance_matrix, 20, heuristic_rollout, 10)

print (distance)

7997


## Fazit

- wir haben Beam Search auf das Rucksack-Problem angewendet
- wir haben die so genannte **informierte Suche** kennengelernt, 
- insbesondere **Rollout** als heuristische Funktion
- und eine Technik, um "teure" Heuristiken wie Rollout nur begrenzt anzuwenden

- wir haben noch etwas Zeit, um Fragen mit den Gruppen zu klären